In [1]:
import os
from urllib.request import urlopen

files = ["eng.testa", "eng.testb", "eng.train"]
url = "https://raw.githubusercontent.com/glample/tagger/master/dataset"
if not os.path.isdir("data"):
    os.mkdir("data")
for file in files:
    with open(f"data/{file}", 'a') as the_file:
        html = urlopen(f"{url}/{file}")
        for line in html:
            the_file.write(line.decode('UTF-8'))

In [2]:
labels = ["O", "B-MISC", "I-MISC", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC"]

In [3]:

import torch

from torch.utils.data import TensorDataset
from transformers import BertTokenizer

import os
import gc
import logging
from random import sample
from urllib.request import urlopen
import numpy as np
import itertools
from tqdm import tqdm, trange
import sklearn as sk
import sklearn.linear_model
import torch
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset

logger = logging.getLogger(__name__)

class InputExample(object):
    """A single training/test example for token classification."""

    def __init__(self, guid, words: list, labels: list = []):
        """Constructs a InputExample.
        Args:
            guid: Unique id for the example.
            words: list. The words of the sequence.
            labels: (Optional) list. The labels for each word of the sequence. This should be
                specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.words = words
        self.labels = labels


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_ids):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_ids = label_ids


def read_examples_from_file(data_dir, mode="eng.testa"):
    """Creating InputExamples out of a file"""
    file_path = os.path.join(data_dir, "{}".format(mode))
    guid_index = 1
    examples = []
    with open(file_path, encoding="utf-8") as f:
        words = []
        labels = []
        for line in f:
            if line.startswith("-DOCSTART-") or line == "" or line == "\n":
                if words:
                    examples.append(InputExample(guid="{}-{}".format(mode, guid_index), words=words, labels=labels))
                    guid_index += 1
                    words = []
                    labels = []
            else:
                splits = line.split(" ")
                words.append(splits[0])
                if len(splits) > 1:
                    labels.append(splits[-1].replace("\n", ""))
                else:
                    # Examples could have no label for mode = "test"
                    labels.append("O")
        if words:
            examples.append(InputExample(guid="{}-{}".format(mode, guid_index), words=words, labels=labels))
    return examples


def convert_examples_to_features(
    examples,
    label_list,
    max_seq_length,
    tokenizer,
    cls_token_at_end=False,
    cls_token="[CLS]",
    cls_token_segment_id=1,
    sep_token="[SEP]",
    sep_token_extra=False,
    pad_on_left=False,
    pad_token=0,
    pad_token_segment_id=0,
    pad_token_label_id=-100,
    sequence_a_segment_id=0,
    mask_padding_with_zero=True,
):
    """ Loads a data file into a list of `InputBatch`s
        `cls_token_at_end` define the location of the CLS token:
            - False (Default, BERT/XLM pattern): [CLS] + A + [SEP] + B + [SEP]
            - True (XLNet/GPT pattern): A + [SEP] + B + [SEP] + [CLS]
        `cls_token_segment_id` define the segment id associated to the CLS token (0 for BERT, 2 for XLNet)
    """

    label_map = {label: i for i, label in enumerate(label_list)}

    features = []
    for (ex_index, example) in enumerate(examples):
        if ex_index % 10000 == 0:
            logger.info("Writing example %d of %d", ex_index, len(examples))

        tokens = []
        label_ids = []
        for word, label in zip(example.words, example.labels):
            word_tokens = tokenizer.tokenize(word)
            tokens.extend(word_tokens)
            # Use the real label id for the first token of the word, and padding ids for the remaining tokens
            label_ids.extend([label_map[label]] + [pad_token_label_id] * (len(word_tokens) - 1))

        # Account for [CLS] and [SEP] with "- 2" and with "- 3" for RoBERTa.
        special_tokens_count = 3 if sep_token_extra else 2
        if len(tokens) > max_seq_length - special_tokens_count:
            tokens = tokens[: (max_seq_length - special_tokens_count)]
            label_ids = label_ids[: (max_seq_length - special_tokens_count)]

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids:   0   0  0    0    0     0       0   0   1  1  1  1   1   1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids:   0   0   0   0  0     0   0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens += [sep_token]
        label_ids += [pad_token_label_id]
        if sep_token_extra:
            # roberta uses an extra separator b/w pairs of sentences
            tokens += [sep_token]
            label_ids += [pad_token_label_id]
        segment_ids = [sequence_a_segment_id] * len(tokens)

        if cls_token_at_end:
            tokens += [cls_token]
            label_ids += [pad_token_label_id]
            segment_ids += [cls_token_segment_id]
        else:
            tokens = [cls_token] + tokens
            label_ids = [pad_token_label_id] + label_ids
            segment_ids = [cls_token_segment_id] + segment_ids

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding_length = max_seq_length - len(input_ids)
        if pad_on_left:
            input_ids = ([pad_token] * padding_length) + input_ids
            input_mask = ([0 if mask_padding_with_zero else 1] * padding_length) + input_mask
            segment_ids = ([pad_token_segment_id] * padding_length) + segment_ids
            label_ids = ([pad_token_label_id] * padding_length) + label_ids
        else:
            input_ids += [pad_token] * padding_length
            input_mask += [0 if mask_padding_with_zero else 1] * padding_length
            segment_ids += [pad_token_segment_id] * padding_length
            label_ids += [pad_token_label_id] * padding_length

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        assert len(label_ids) == max_seq_length

        if ex_index < 5:
            logger.info("*** Example ***")
            logger.info("guid: %s", example.guid)
            logger.info("tokens: %s", " ".join([str(x) for x in tokens]))
            logger.info("input_ids: %s", " ".join([str(x) for x in input_ids]))
            logger.info("input_mask: %s", " ".join([str(x) for x in input_mask]))
            logger.info("segment_ids: %s", " ".join([str(x) for x in segment_ids]))
            logger.info("label_ids: %s", " ".join([str(x) for x in label_ids]))

        features.append(
            InputFeatures(input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids, label_ids=label_ids)
        )
    return features

tokenizer = BertTokenizer.from_pretrained("bert-base-cased")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [24]:
from transformers import BertModel
bert_model = BertModel.from_pretrained("bert-base-cased")

for param in bert_model.embeddings.parameters():
    param.requires_grad = False

In [5]:
dataset = "eng.train"
examples = read_examples_from_file(".", mode=f"data/{dataset}")
pad_token_label_id = torch.nn.CrossEntropyLoss().ignore_index

features = convert_examples_to_features(
    examples,
    label_list=labels,
    max_seq_length=128,
    tokenizer=tokenizer,
    cls_token_at_end=False,
    cls_token=tokenizer.cls_token,
    cls_token_segment_id=0,
    sep_token=tokenizer.sep_token,
    sep_token_extra=False,
    pad_on_left=False,
    pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
    pad_token_segment_id=0,
    pad_token_label_id=pad_token_label_id)

all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
all_label_ids = torch.tensor([f.label_ids for f in features], dtype=torch.long)

tensordataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)


dataset_test = "eng.testa"
test_examples = read_examples_from_file(".", mode=f"data/{dataset_test}")

test_features = convert_examples_to_features(
    test_examples,
    label_list=labels,
    max_seq_length=128,
    tokenizer=tokenizer,
    cls_token_at_end=False,
    cls_token=tokenizer.cls_token,
    cls_token_segment_id=0,
    sep_token=tokenizer.sep_token,
    sep_token_extra=False,
    pad_on_left=False,
    pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
    pad_token_segment_id=0,
    pad_token_label_id=pad_token_label_id)


all_input_ids = torch.tensor([f.input_ids for f in test_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in test_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in test_features], dtype=torch.long)
all_label_ids = torch.tensor([f.label_ids for f in test_features], dtype=torch.long)

test_tensordataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)

In [6]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

eval_sampler = SequentialSampler(tensordataset)
eval_dataloader = DataLoader(tensordataset, sampler=eval_sampler, batch_size=1)

eval_sampler = SequentialSampler(test_tensordataset)
test_eval_dataloader = DataLoader(test_tensordataset, sampler=eval_sampler, batch_size=16)

In [26]:
import torch
import torch.nn as nn

class Embeddings(nn.Module):
    def __init__(self, vocab_size, embedding_dim, max_length):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.position_embedding = nn.Embedding(max_length, embedding_dim)
        self.max_length = max_length

    def forward(self, input_ids):

        token_embeddings = self.embedding(input_ids)

        position_ids = torch.arange(self.max_length).unsqueeze(0).repeat(input_ids.size(0), 1).to(input_ids.device)

        position_embeddings = self.position_embedding(position_ids)
        embeddings = token_embeddings + position_embeddings

        return embeddings


class LSTMCell(nn.Module):

    def __init__(self, input_lenght, hidden_lenght):
        super(LSTMCell, self).__init__()
        self.input_lenght = input_lenght
        self.hidden_lenght = hidden_lenght

        # Forget Gate
        self.forget_w1 = nn.Linear(self.input_lenght, self.hidden_lenght, bias=True)
        self.forget_r1 = nn.Linear(self.hidden_lenght, self.hidden_lenght, bias=False)
        self.forget_sigmoid = nn.Sigmoid()

        # Input Gate
        self.input_w2 = nn.Linear(self.input_lenght, self.hidden_lenght, bias=True)
        self.input_r2 = nn.Linear(self.hidden_lenght, self.hidden_lenght, bias=False)
        self.input_sigmoid = nn.Sigmoid()

        # Cell Memory
        self.memory_w3 = nn.Linear(self.input_lenght, self.hidden_lenght, bias=True)
        self.memory_r3 = nn.Linear(self.hidden_lenght, self.hidden_lenght, bias=False)
        self.memory_activation = nn.Tanh()

        # Output Gate
        self.output_w4 = nn.Linear(self.input_lenght, self.hidden_lenght, bias=True)
        self.output_r4 = nn.Linear(self.hidden_lenght, self.hidden_lenght, bias=False)
        self.output_sigmoid = nn.Sigmoid()
        self.final_activation = nn.Tanh()

    def forget_gate(self, x, h):
        return self.forget_sigmoid(self.forget_w1(x)+self.forget_r1(h))

    def input_gate(self, x, h):
        return self.input_sigmoid(self.input_w2(x)+self.input_r2(h))

    def memory_gate(self, i, f, x, h, c_prev):
        g = self.memory_activation(self.memory_w3(x)+self.memory_r3(h)) * i
        c = f * c_prev
        return g + c


    def output_gate(self, x, h):
        return self.output_sigmoid(self.output_w4(x) + self.output_r4(h))

    def forward(self, x, input):
        (h, c_prev) = input
        # Input
        i = self.input_gate(x, h)
        # Forget
        f = self.forget_gate(x, h)
        # Update memory
        c_next = self.memory_gate(i, f, x, h, c_prev)
        # Output
        o = self.output_gate(x, h)
        h_next = o * self.final_activation(c_next)

        return h_next, c_next


class LSTM(nn.Module):
    def __init__(self, vocab_size = 100, seq_size = 128, input_shape=768, hidden_shape=64, num_class=9, embeddings = None, cell_number=3):
        super().__init__()

        if embeddings is None:
            self.embedding = Embeddings(vocab_size=vocab_size,
                                              embedding_dim=input_shape,
                                              max_length=seq_size)
        else:
            self.embedding = embeddings

        self.rnn1 = LSTMCell(input_shape, hidden_shape)
        self.rnn2 = LSTMCell(hidden_shape, hidden_shape)
        self.linear = nn.Linear(hidden_shape, num_class)
        self.hidden_shape = hidden_shape

    def forward(self, input):
        outputs = []

        embeds = self.embedding(input)

        h_t = torch.zeros(input.size(0), self.hidden_shape)
        c_t = torch.zeros(input.size(0), self.hidden_shape)
        h_t2 = torch.zeros(input.size(0), self.hidden_shape)
        c_t2 = torch.zeros(input.size(0), self.hidden_shape)

        # h_t = torch.zeros(input.size(0), self.hidden_shape).to("cuda")
        # c_t = torch.zeros(input.size(0), self.hidden_shape).to("cuda")
        # h_t2 = torch.zeros(input.size(0), self.hidden_shape).to("cuda")
        # c_t2 = torch.zeros(input.size(0), self.hidden_shape).to("cuda")


        for i in range(embeds.size(1)):

            # if self.LSTM:
            h_t, c_t = self.rnn1(embeds[:, i, :], (h_t, c_t))
            h_t2, c_t2 = self.rnn2(h_t, (h_t2, c_t2))
            # else:
            #     h_t = self.rnn1(input_t, h_t)
            #     h_t2 = self.rnn2(h_t, h_t2)


            output = self.linear(h_t2)
            outputs += [output]

        outputs = torch.stack(outputs, 1).squeeze(2)
        return outputs


In [27]:

model = LSTM(vocab_size=len(tokenizer.vocab), embeddings=bert_model.embeddings)

total_params = sum(p.numel() for p in model.parameters())
print(f'Total parameters: {total_params}')

Total parameters: 22912073


In [28]:
import torch

from tqdm.auto import tqdm
from typing import Dict, List, Tuple

pad_token_label_id = torch.nn.CrossEntropyLoss().ignore_index

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
    model.train()

    train_loss, train_acc = 0, 0
    for batch_num, batch, in enumerate(dataloader):

        inputs = {"input": batch[0].to(device)}

        y_pred = model(**inputs)[0].unsqueeze(dim=0)
        preds = y_pred[0, (batch[3] != pad_token_label_id)[0], :]
        y = (batch[3][0][(batch[3] != pad_token_label_id)[0]]).to(device)

        loss = loss_fn(preds, y)
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        y_pred_class = torch.argmax(torch.softmax(preds, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred_class.view(-1))
        if batch_num % 15 == 0:
            print("accuracy: ", train_acc/(batch_num+1), "loss: ", train_loss/(batch_num+1), "batch: ", batch_num)
            # print((y_pred_class == y).sum().item()/len(y_pred_class.view(-1)))

    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
    model.eval()

    test_loss, test_acc = 0, 0

    with torch.inference_mode():
        for batch_num, batch in enumerate(dataloader):

            inputs = {"input": batch[0].to(device)}

            y_pred = model(**inputs)[0].unsqueeze(dim=0)
            preds = y_pred[0, (batch[3] != pad_token_label_id)[0], :]
            y = (batch[3][0][(batch[3] != pad_token_label_id)[0]]).to(device)


            test_pred_labels = preds.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels.view(-1)))

    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:

    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }

    model.to(device)

    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    return results


In [29]:
optimizer = torch.optim.Adam(params=model.parameters(),
                             lr=3e-3,
                             betas=(0.9, 0.999),
                             weight_decay=0.3)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

epochs = 1

# print(train_step(model, eval_dataloader, torch.nn.CrossEntropyLoss(), optimizer, device))
results = train(model, eval_dataloader, test_eval_dataloader, optimizer, torch.nn.CrossEntropyLoss(), epochs, device)

  0%|          | 0/1 [00:00<?, ?it/s]

accuracy:  0.4444444444444444 loss:  2.1185905933380127 batch:  0
accuracy:  0.7643526931934189 loss:  2.0011844858527184 batch:  15
accuracy:  0.7919784102320676 loss:  1.7693296401731429 batch:  30
accuracy:  0.8019073648127583 loss:  1.4796959220067314 batch:  45
accuracy:  0.8094419796351268 loss:  1.3056238327847152 batch:  60
accuracy:  0.8134619846038527 loss:  1.2330384960300045 batch:  75
accuracy:  0.8162663563134109 loss:  1.1730298786372928 batch:  90
accuracy:  0.8080555874591907 loss:  1.1584539840806205 batch:  105
accuracy:  0.817759246799253 loss:  1.1007214922550297 batch:  120
accuracy:  0.8070559730043946 loss:  1.1052846268695944 batch:  135
accuracy:  0.809084036342607 loss:  1.0791737608956975 batch:  150
accuracy:  0.8015315083100344 loss:  1.0803195667195034 batch:  165
accuracy:  0.7999156807189408 loss:  1.0695430237943955 batch:  180
accuracy:  0.7997782222772243 loss:  1.059694406329369 batch:  195
accuracy:  0.7889147552561754 loss:  1.0708584226138218 bat

KeyboardInterrupt: 

In [ ]:
print(test_step(model=model,
          dataloader=test_eval_dataloader,
          loss_fn=torch.nn.CrossEntropyLoss(),
          device=device))

(0.0, 0.7762199781104382)
